<h1 style="font-size:42px; text-align:center; margin-bottom:30px;"><span style="color:SteelBlue">Module 4:</span> Model Training</h1>
<hr>

At last, it's time to build our models! 

Remember, professional data scientists actually spend the bulk of their time on the 3 steps leading up to this one: 
1. Exploratory Analysis
2. Data Cleaning
3. Feature Engineering

Again, that's because **better data beats fancier algorithms**.

<br><hr id="toc">

### In this module...

First, we'll load our analytical base table from Module 2. 

Then, we'll go through the essential modeling steps:

1. [Split your dataset](#split)
2. [Build model pipelines](#pipelines)
3. [Declare hyperparameters to tune](#hyperparameters)
4. [Fit and tune models with cross-validation](#fit-tune)
5. [Evaluate metrics](#evaluate)
6. [Area under ROC curve](#auroc)

Finally, we'll save the best model to use in the next module. Since we've seen these steps in Project 2 already, we'll try to speed through most of them. However, we'll spend a little extra time on the last 2 steps because classification tasks require new metrics.

<br><hr>

### First, let's import libraries, recruit models, and load the ABT.

First, let's import the libraries that we'll need.

In [ ]:
# print_function for compatibility with Python 3

# NumPy for numerical computing

# Pandas for DataFrames


# Matplotlib for visualization

# display plots in the notebook


# Seaborn for easier visualization

# Scikit-Learn for Modeling

# Pickle for saving model files


Next, let's import the algorithms we introduced in Module 3.

In [ ]:
# Import Logistic Regression

# Import RandomForestClassifier and GradientBoostingClassifer


Next, let's import the Scikit-Learn functions and helpers we'll need for this module.
* Remember, it's OK to just add more imports here as you need them and re-run this cell (it won't change anything else you've done).

In [ ]:
# Function for splitting training and test set

# Function for creating model pipelines

# For standardization

# Helper for cross-validation

# Classification metrics (added later)


Finally, let's read the analytical base table you saved at the end of Module 2.

In [ ]:
# Load analytical base table from Module 2


<span id="split"></span>
# 1. Split your dataset

Just as we did in Project 2, let's start by splitting our data into separate training and test sets. 

<br>
**First, separate the dataframe into separate objects for the target variable, <code style="color:steelblue">y</code>, and the input features, <code style="color:steelblue">X</code>.**

In [ ]:
# Create separate object for target variable


# Create separate object for input features


**After you've imported the <code style="color:steelblue">train_test_split()</code> function, split <code style="color:steelblue">X</code> and <code style="color:steelblue">y</code> into training and test sets.**
* Pass in the argument <code style="color:steelblue">test_size=<span style="color:crimson">0.2</span></code> to set aside 20% of our observations for the test set.
* Pass in <code style="color:steelblue">random_state=<span style="color:crimson">1234</span></code> to set the random state for replicable results.
* **Important:** Also pass in the argument <code style="color:steelblue">stratify=<span style="color:crimson">df.status</span></code> in order to make sure the target variable's classes are balanced in each subset of data! This is **stratified random sampling**.
* Then, print the number of observations in each subset to check that it was done correctly.

In [ ]:
# Split X and y into train and test sets


# Print number of observations in X_train, X_test, y_train, and y_test


<p style="text-align:center; margin: 40px 0 40px 0; font-weight:bold">
<a href="#toc">Back to Contents</a>
</p>

<span id="pipelines"></span>
# 2. Build model pipelines

Next, let's set up preprocessing pipelines for each of our algorithms.

<br>
**Create a single <span style="color:royalblue">pipeline dictionary</span> with pipelines for each algorithm**.
* Use the keys:
    * <code style="color:crimson">'l1'</code> for $L_1$-regularized logistic regression
    * <code style="color:crimson">'l2'</code> for $L_2$-regularized logistic regression
    * <code style="color:crimson">'rf'</code> for random forest
    * <code style="color:crimson">'gb'</code> for gradient boosted tree.
* Each pipeline should standardize the data first.
* Remember to set <code style="color:steelblue">random_state=<span style="color:crimson">123</span></code> for each algorithm to ensure replicable results.

In [ ]:
# Pipeline dictionary


<p style="text-align:center; margin: 40px 0 40px 0; font-weight:bold">
<a href="#toc">Back to Contents</a>
</p>

<span id="hyperparameters"></span>
# 3. Declare hyperparameters to tune

Next, let's declare hyperparameters to tune.

<br>
**First, list the tunable hyperparameters of your $L_1$-regularized logistic regression pipeline.**

In [ ]:
# List tuneable hyperparameters of our Logistic pipeline


Let's declare the **hyperparameter grids** to tune.

In [ ]:
# Logistic Regression hyperparameters


**Declare the hyperparameter grid for the random forest.**

In [ ]:
# Random Forest hyperparameters


**Declare the hyperparameter grid for the boosted tree.**

In [ ]:
# Boosted Tree hyperparameters


**Create a <code style="color:steelblue">hyperparameters</code> dictionary**.
* Use the same keys as in the <code style="color:steelblue">pipelines</code> dictionary.

In [ ]:
# Create hyperparameters dictionary


<p style="text-align:center; margin: 40px 0 40px 0; font-weight:bold">
<a href="#toc">Back to Contents</a>
</p>

<span id="fit-tune"></span>
# 4. Fit and tune models with cross-validation

Now that we have our <code style="color:steelblue">pipelines</code> and <code style="color:steelblue">hyperparameters</code> dictionaries declared, we're ready to tune our models with **cross-validation**.

<br>
**Create a <code style="color:SteelBlue">fitted_models</code> dictionary that includes models that have been tuned using cross-validation.**
* The keys should be the same as those in the <code style="color:SteelBlue">pipelines</code> and <code style="color:SteelBlue">hyperparameters</code> dictionaries. 
* The values should be <code style="color:steelblue">GridSearchCV</code> objects that have been fitted to <code style="color:steelblue">X_train</code> and <code style="color:steelblue">y_train</code>.
* After fitting each model, print <code style="color:crimson">'{name} has been fitted.'</code> just to track the progress.
* (Optionally) You can set <code style="color:steelblue">n_jobs=<span style="color:crimson">-1</span></code> to use as many cores as available on your computer.

This step can take a few minutes, so please be patient.

In [ ]:
# Create empty dictionary called fitted_models


# Loop through model pipelines, tuning each one and saving it to fitted_models

    # Create cross-validation object from pipeline and hyperparameters
    
    
    # Fit model on X_train, y_train
    
    
    # Store model in fitted_models[name] 
    
    
    # Print '{name} has been fitted'


<span id="evaluate"></span>
# 5. Evaluate metrics

Finally, it's time to evaluate our models and pick the best one.

<br>
**First, display the <code style="color:steelblue">best\_score_</code> attribute for each fitted model.**

In [ ]:
# Display best_score_ for each fitted model


If you get different numbers, check to see that you've set the <code style="color:steelblue">random_state=</code> correctly for each of the models.

<p style="text-align:center; margin: 40px 0 40px 0; font-weight:bold">
<a href="#toc">Back to Contents</a>
</p>

<span id="auroc"></span>
# 6. Area under ROC curve

**Area under ROC curve** is the most reliable metric for classification tasks.

<br>
**Add this import to the top of your Companion Workbook (good practice for keeping all your imports in one place.** 
* Remember to run the entire code cell afterward.

In [ ]:
# Classification metrics
from sklearn.metrics import roc_curve, auc

Before presenting the idea of an ROC curve, we must first discuss what a **confusion matrix** is. 

Let's see an example using our $L_1$-regularized logistic regression. 
* First, let's use <code style="color:steelblue">.predict()</code> to get the predicted classes directly.

In [ ]:
# Predict classes using L1-regularized logistic regression 


# Display first 5 predictions


* Next, let's display the confusion matrix:

In [ ]:
# Import confusion_matrix


# Display confusion matrix for y_test and pred


Remember, we can predict a **probability** for each class using <code style="color:steelblue">.predict_proba()</code>, instead of the class directly.

For example:

In [ ]:
# Predict PROBABILITIES using L1-regularized logistic regression


# Get just the prediction for the positive class (1)


# Display first 5 predictions


Let's calculate the ROC curve using the <code style="color:steelblue">roc_curve()</code> function that we imported earlier.

In [ ]:
# Calculate ROC curve from y_test and pred


We can throw these into a DataFrame for convenience and look at the last 10:

In [ ]:
# Store fpr, tpr, thresholds in DataFrame and display last 10


As you can see, as you decrease the threshold, both the false positive rate **and** the true positive rate increase.

We can plot the entire curve.

In [ ]:
# Initialize figure


# Plot ROC curve


# Diagonal 45 degree line


# Axes limits and labels



To calculate AUROC, use the <code style="color:steelblue">auc()</code> function we imported earlier in conjunction with the <code style="color:steelblue">roc_curve()</code> function.

In [ ]:
# Calculate ROC curve

# Calculate AUROC


Now that we've taken a detour to dive into some of the intuition behind AUROC, let's calculate it for each of our fitted models on the test set.

<br>
**Using a <code style="color:SteelBlue">for</code> loop, print the performance of each model in <code style="color:SteelBlue">fitted_models</code> on the test set.**
* Print the <code style="color:SteelBlue">auc</code> of the <code style="color:SteelBlue">roc_curve</code>.
* Label the output with the name of the algorithm. For example:

<pre style="color:crimson">
rf 0.991520189216
</pre>

In [ ]:
# Code here



**Finally, save the winning <code style="color:steelblue">Pipeline</code> object into a pickle file.**
* Just save the <code style="color:steelblue">Pipeline</code>, not the <code style="color:steelblue">GridSearchCV</code> object.
* **Hint:** Remember the <code style="color:steelblue">.best\_estimator_</code> attribute?

In [ ]:
# Save winning model as final_model.pkl


<p style="text-align:center; margin: 40px 0 40px 0; font-weight:bold">
<a href="#toc">Back to Contents</a>
</p>

<br>
## Next Steps

Congratulations for making through Project 3's Model Training module!

As a reminder, here are a few things you did in this module:
* You split your dataset into training and test sets.
* You set up model pipelines and hyperparameter grids.
* You tuned your models using cross-validation.
* You learned about how AUROC is a more effective metric for classification than simple accuracy.
* And finally, you saved the winning model.

In the next module, <span style="color:royalblue">Module 5: Project Delivery</span>, we'll see how you can go the extra mile in terms of project delivery. That includes preparing to apply your model to raw data and packaging it up into an executable script.

<p style="text-align:center; margin: 40px 0 40px 0; font-weight:bold">
<a href="#toc">Back to Contents</a>
</p>